In [ ]:
import string
import numpy
import requests
import urllib.request
import json
from bs4 import BeautifulSoup
import re
import itertools
import copy
import csv
import json
from pprint import pprint
from sklearn import datasets, linear_model


In [ ]:
PATH_TO_DATA = "C:/Users/mikeb/Documents/Code/CSE258/data/project/"
PATH_TO_RESULTS = "C:/Users/mikeb/Documents/Code/CSE258/data/project/results/"
URL = "http://ws.audioscrobbler.com/2.0/"
API_KEY = "465ac27dd83752fc45924159f97ef62f"
HEADERS = {
    'content-type': "application/json",
    'cache-control': "no-cache"
    }

In [ ]:
# read albums from CSV
def parseCSVData(fname):
    with open(PATH_TO_DATA + fname, encoding="utf8") as f:
        src = itertools.islice(f, 1, None) # skip header
        reader = csv.reader(src,delimiter=',')
        return list(reader)
    
charts = parseCSVData("charts.csv")
charts = charts[:2000] # only use top 2,000 albums

In [ ]:
# process albums to map to artists
data = {}
for album in charts:
    album_rank = int(album[0])
    album_name = album[1]
    artist = album[2]
            
    # trim so it only has two artist
#     if (artist == "Queen" or artist == "Pink Floyd"):
    val = {"name" : album_name, "rank" : album_rank}
    if artist in data:
        data[artist]["albums"].append(val)
    else:
        data[artist] = {"albums" : [val]}

In [ ]:
def getSongsFromArtistAlbum(artist, album):
    songs = []
    try:
        querystring = {"method":"album.getinfo","api_key": API_KEY,"artist":artist,"album":album,"format":"json"}
        payload = "{ \n\n}"
        response = requests.request("POST", URL, data=payload, headers=HEADERS, params=querystring)
        tracks = response.json()['album']['tracks']['track']
        for track in tracks:
            songs.append({"name" : track['name']})
    except:
        songs = []
    return songs

# getSongsFromArtistAlbum("Pink Floyd","The Dark Side Of The Moon")

In [ ]:
# get songs for all the albums
total_number_of_songs = 0
for artist in data:
    print ("mapping songs to albums for ", artist)
    for album in data[artist]["albums"]:
        songs = getSongsFromArtistAlbum(artist,album["name"])
        total_number_of_songs += len(songs)
        album["songs"] = songs
        
print("total # of songs mapped ", total_number_of_songs)

In [ ]:
for artist in data:
    for album in data[artist]["albums"]:
        if len(album["songs"])  == 0: 
            print(artist, "\t", album)

In [ ]:
def processStringForURL(val):
    val = val.replace(" ", "-")
    val = val.replace("'", "")
    val = val.replace("?", "")
    val = val.replace("!", "")
    val = val.replace(".", "")
    val = val.replace(",", "")
    return val

def getLyrics(artist, song):
    
    lyrics_url = "https://genius.com/" + processStringForURL(artist) + "-" + processStringForURL(song) + "-lyrics"
#     print(lyrics_url)
    lyrics = ""
    try:
        page = requests.get(lyrics_url)
        soup = BeautifulSoup(page.text, "html.parser")
        lyrics = soup.find(class_='lyrics').text
    except:
        lyrics = ""
  
    return lyrics

# getLyrics("Pink Floyd", "Wish You Were Here")


In [ ]:
# map lyrics to songs
count_success = 0
count_fail = 0
for artist in data:
    print("mapped artist ", artist)
    for album in data[artist]["albums"]:
        for song in album["songs"]:
            lyrics = getLyrics(artist, song["name"])
            if (len(lyrics) == 0):
                count_fail += 1
            else:
                count_success += 1
            song["lyrics"] = lyrics

In [ ]:
success_rate = count_success / (count_success + count_fail)
print("Success Rate: ", success_rate)

In [ ]:
# for albums in data["Queen"]["albums"]:
#     for songs in albums["songs"]:
#         if (len(songs["lyrics"]) == 0):
#             print (songs["name"])

In [ ]:
# dump results
for artist in data:
    with open(PATH_TO_RESULTS + artist + ".json", 'w', encoding="utf8") as f:
        json.dump(data[artist], f, ensure_ascii=False)

In [ ]:
pprint(data)